In [1]:
import pandas as pd

import torch

from experiments import benchmark_experiment
from utils import get_data

torch.set_grad_enabled(False)

device = "cuda" if torch.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2

/home/jgcarrasco/.virtualenvs/mech_interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# acronyms -> 250
# ioi -> 150
# greater-than -> 250
n_patching = 250
n_val = 250
task = "greater-than"
threshold = 0.085317
include_mlps = True

In [3]:
rows = []

data = get_data(n_patching=n_patching, n_val=n_val, task=task)

patching_tokens = data["patching_tokens"].cuda()
patching_cache = data["patching_cache"]

val_tokens = data["val_tokens"].cuda()
val_answer_tokens = data["val_answer_tokens"].cuda()
val_logits = data["val_logits"]

acc, d_acc, size, d_size, time, d_time = benchmark_experiment(threshold, val_logits, val_tokens, val_answer_tokens, 
                                                            patching_tokens, patching_cache, task, ablation_scheme="mean", 
                                                            include_mlps=include_mlps)
rows.append([acc, d_acc, size, d_size, time, d_time])
df = pd.DataFrame(rows, columns=["Accuracy", "D Acc", "Size", "D Size", "Time", "D time"])

Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
try:
    df_total = pd.read_csv(f"benchmark_results/{task}_{threshold:.2f}{'_mlp' if include_mlps else ''}.csv")
    df_total = pd.concat([df_total, df])
except:
    df_total = df

df_total.to_csv(f"benchmark_results/{task}_{threshold:.2f}{'_mlp' if include_mlps else ''}.csv", index=False)

In [5]:
df_total

,Accuracy,D Acc,Size,D Size,Time,D time
0,1.000,0.000,14770752,0.826341,0.001037,0.871389
1,1.000,0.000,14770752,0.826341,0.001022,0.873376
2,0.992,-0.008,14571648,0.828682,0.000888,0.890042
3,1.000,0.000,14571648,0.828682,0.000882,0.890711
0,1.000,0.004,14571648,0.828682,0.000876,0.891358


In [6]:
df_total.describe()

,Accuracy,D Acc,Size,D Size,Time,D time
count,5.000000,5.000000,5.000000e+00,5.000000,5.000000,5.000000
mean,0.998400,-0.000800,1.465129e+07,0.827745,0.000941,0.883375
std,0.003578,0.004382,1.090538e+05,0.001282,0.000081,0.010070
min,0.992000,-0.008000,1.457165e+07,0.826341,0.000876,0.871389
25%,1.000000,0.000000,1.457165e+07,0.826341,0.000882,0.873376
50%,1.000000,0.000000,1.457165e+07,0.828682,0.000888,0.890042
75%,1.000000,0.000000,1.477075e+07,0.828682,0.001022,0.890711
max,1.000000,0.004000,1.477075e+07,0.828682,0.001037,0.891358
